<a href="https://colab.research.google.com/github/MendesAdriano/CyberNewsClassification/blob/main/finetune_secBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning SecBERT LM for text classification
Mount your own drive space as working space with the following three commands



In [1]:
import pandas as pd

In [6]:
df = pd.read_excel('/content/TheHackerNews_Dataset.xlsx')

In [8]:
df.shape


(3742, 4)

In [10]:
df2 = df.drop_duplicates(subset='Title', keep='first')
df2.shape

(3223, 4)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!cd '/content/drive/MyDrive/secBert'

Upload into the secBert Directory in your drive space dataset's files

In [13]:
import os
os.chdir('/content/drive/MyDrive/secBert')

In [14]:
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
!nvcc --version
!nvidia-smi

ls: cannot access '/var/cuda-repo-9-0-local': No such file or directory
gpg: can't open '/var/cuda-repo-9-0-local/7fa2af80.pub': No such file or directory
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,016 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,165 kB]
Get:11 http://security.

In [15]:
!pip install pandas
!pip3 install torch torchvision
!pip install transformers
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [16]:
import torch
import pandas as pd

from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig, AutoModel
from sklearn.preprocessing import LabelEncoder

from time import sleep

In [17]:
tokenizer = AutoTokenizer.from_pretrained("jackaduma/SecBERT")

pretrained_model = AutoModelForMaskedLM.from_pretrained("jackaduma/SecBERT")
config = BertConfig.from_pretrained("jackaduma/SecBERT", output_hidden_states=True)

Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Set the Runtime to GPU and check and set cuda availability with the following snippet

In [18]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

Substitute dataset file name with your own

In [13]:
df = pd.read_excel('/content/drive/MyDrive/secBert/TheHackerNews_Dataset.xlsx') #Change name here
df

,Title,Link,Article,Label
0,Critical Cosmos Database Flaw Affected Thousan...,https://thehackernews.com/2021/08/critical-cos...,Cloud infrastructure security company Wiz on T...,Data_Breaches
1,Researchers Detail Modus Operandi of ShinyHunt...,https://thehackernews.com/2021/08/researchers-...,"ShinyHunters, a notorious cybercriminal underg...",Data_Breaches
2,Chinese Hackers Believed to be Behind Second C...,https://thehackernews.com/2021/06/chinese-hack...,Even as a massive data breach affecting Air In...,Data_Breaches
3,Using Breached Password Detection Services to ...,https://thehackernews.com/2021/06/using-breach...,Bolstering password policies in your organizat...,Data_Breaches
4,Can Data Protection Systems Prevent Data At Re...,https://thehackernews.com/2021/05/can-data-pro...,Protection against insider risks works when th...,Data_Breaches
...,...,...,...,...
3737,Chinese hackers attack on White House computers,https://thehackernews.com/2012/10/chinese-hack...,The White House acknowledged Monday that one o...,Cyber_Attack
3738,Cyber Attacks on Six Major American Banks,https://thehackernews.com/2012/10/cyber-attack...,"According to reports, some of the United State...",Cyber_Attack
3739,9 million PCs infected with ZeroAccess botnet,https://thehackernews.com/2012/09/9-million-pc...,"In recent months, we've seen the rootkit famil...",Cyber_Attack
3740,Bharatiya Janata Party website defaced by Anon...,https://thehackernews.com/2012/09/bharatiya-ja...,Hackers from Indian Anonymous Group hacked Bha...,Cyber_Attack


In [ ]:
# df = df.reset_index()

In [19]:
df = df2.copy()

In [20]:
LABELS = len(df['Label'].value_counts())

In [21]:
#Encoding labels
encoder = LabelEncoder()
encoder.fit(df['Label'])
df['enc_label'] = encoder.transform(df['Label'])

In [22]:
df

,Title,Link,Article,Label,enc_label
0,Critical Cosmos Database Flaw Affected Thousan...,https://thehackernews.com/2021/08/critical-cos...,Cloud infrastructure security company Wiz on T...,Data_Breaches,1
1,Researchers Detail Modus Operandi of ShinyHunt...,https://thehackernews.com/2021/08/researchers-...,"ShinyHunters, a notorious cybercriminal underg...",Data_Breaches,1
2,Chinese Hackers Believed to be Behind Second C...,https://thehackernews.com/2021/06/chinese-hack...,Even as a massive data breach affecting Air In...,Data_Breaches,1
3,Using Breached Password Detection Services to ...,https://thehackernews.com/2021/06/using-breach...,Bolstering password policies in your organizat...,Data_Breaches,1
4,Can Data Protection Systems Prevent Data At Re...,https://thehackernews.com/2021/05/can-data-pro...,Protection against insider risks works when th...,Data_Breaches,1
...,...,...,...,...,...
3734,Anonymous threatened Estonian government with ...,https://thehackernews.com/2012/10/anonymous-th...,Anonymous Hackers AnonSwedenOp posted a video ...,Cyber_Attack,0
3735,Swedish authorities raid on PRQ prompts new cy...,https://thehackernews.com/2012/10/swedish-auth...,Computer hackers claiming to be from the Anony...,Cyber_Attack,0
3737,Chinese hackers attack on White House computers,https://thehackernews.com/2012/10/chinese-hack...,The White House acknowledged Monday that one o...,Cyber_Attack,0
3740,Bharatiya Janata Party website defaced by Anon...,https://thehackernews.com/2012/09/bharatiya-ja...,Hackers from Indian Anonymous Group hacked Bha...,Cyber_Attack,0


In [23]:
df = df[['Label', 'Title', 'enc_label']]
LABELS

4

In [24]:
AVG_LEN = df['Title'].str.len().mean()
STD_LEN = df['Title'].str.len().std()
print(AVG_LEN,STD_LEN)


66.1160409556314 11.26106050789373


In [52]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 16
EPOCHS = 9
LEARNING_RATE = 1e-05


In [53]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        Title = str(self.data.Title[index])
        Title = " ".join(Title.split())
        inputs = self.tokenizer.encode_plus(
            Title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        if 'enc_label' not in self.data:
            return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
            }

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.enc_label[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [39]:
df.reset_index(inplace=True)

Choose an appropriate name for saving your train and test datasets

In [40]:
df

,index,Label,Title,enc_label
0,0,Data_Breaches,Critical Cosmos Database Flaw Affected Thousan...,1
1,1,Data_Breaches,Researchers Detail Modus Operandi of ShinyHunt...,1
2,2,Data_Breaches,Chinese Hackers Believed to be Behind Second C...,1
3,3,Data_Breaches,Using Breached Password Detection Services to ...,1
4,4,Data_Breaches,Can Data Protection Systems Prevent Data At Re...,1
...,...,...,...,...
3218,3734,Cyber_Attack,Anonymous threatened Estonian government with ...,0
3219,3735,Cyber_Attack,Swedish authorities raid on PRQ prompts new cy...,0
3220,3737,Cyber_Attack,Chinese hackers attack on White House computers,0
3221,3740,Cyber_Attack,Bharatiya Janata Party website defaced by Anon...,0


In [31]:
from sklearn.model_selection import train_test_split

In [41]:
#Split dataset into train and validation
train_indices, test_indices = train_test_split(list(range(len(df.enc_label))), test_size=0.2, stratify=df.enc_label)

In [42]:
train_dataset = df.copy().drop(test_indices).reset_index(drop=True)
test_dataset = df.copy().drop(train_indices).reset_index(drop=True)

test_dataset.to_csv('test_dataset_mixed_new.csv') #Change name here
train_dataset.to_csv('train_dataset_mixed_new.csv')

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

FULL Dataset: (3223, 4)
TRAIN Dataset: (2578, 4)
TEST Dataset: (645, 4)


In [43]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class SecBERTClass(torch.nn.Module):
    def __init__(self, pretrained_model_name: str, num_classes: int = None, dropout: float = 0.3):
        super().__init__()
        config = BertConfig.from_pretrained(pretrained_model_name, output_hidden_states=True)
        self.model = AutoModel.from_pretrained(pretrained_model_name, config=config).base_model #pick only the main body of the model
        #for param in self.model.parameters():
          #param.requires_grad = False
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(dropout)
        self.classifier = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        output_1 = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [44]:
#LOAD
model = SecBERTClass("jackaduma/SecBERT", LABELS)

In [45]:
model.to(device)

SecBERTClass(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [46]:
len(train_dataset)

2578

In [47]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [54]:
torch.cuda.empty_cache()

# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    examples = len(train_dataset)
    losses = [None] * len(training_loader)
    model.train()
    #loop = tqdm(enumerate(training_loader), total=len(training_loader), leave=False)
    for i, data in enumerate(training_loader, 0):
        #print(i)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)

        losses[i] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
    return


for epoch in range(EPOCHS):
    train(epoch)

Cost at epoch 0 is 0.02546
Cost at epoch 1 is 0.01804
Cost at epoch 2 is 0.01775
Cost at epoch 3 is 0.01270
Cost at epoch 4 is 0.00519
Cost at epoch 5 is 0.00978
Cost at epoch 6 is 0.01068
Cost at epoch 7 is 0.02214
Cost at epoch 8 is 0.01925


In [55]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
      for i, data in enumerate(loader, 0):
          x = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          y = data['targets'].to(device, dtype = torch.long)

          scores = model(x, mask)
          _, predictions = scores.max(1)
          num_correct += (predictions == y).sum()
          num_samples += predictions.size(0)

      print(
          f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
      )



Save the model for further tests

In [56]:
model_path = '.model_mixed.pt'
#SAVE
torch.save(model.state_dict(), model_path)

In [57]:
check_accuracy(testing_loader, model)

Got 490 / 645 with accuracy 75.97
